In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from utils import RatingDataset, train, evaluation
from GMF import GMF
from MLP import MLP
from NCF import NeuralCF  # make sure this points to your NeuralCF definition

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 0.001
n_users, n_items = 943, 1682
gmf_factors = 8
mlp_layers = [32, 16, 8]
epochs = 10

for i in range(1, 6):
    print(f"\n📂 Fold {i}")

    train_path = f'../../data/ml-100k/u{i}.base'
    test_path = f'../../data/ml-100k/u{i}.test'

    train_dataset = RatingDataset(train_path)
    test_dataset = RatingDataset(test_path)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # === PRETRAIN GMF ===
    gmf = GMF(n_users, n_items, gmf_factors).to(device)
    gmf_optimizer = torch.optim.Adam(gmf.parameters(), lr=learning_rate)
    gmf_criterion = nn.MSELoss()

    print("🔧 Pretraining GMF...")
    train(gmf, train_loader, gmf_criterion, gmf_optimizer, device)
    loss, rmse = evaluation(gmf, test_loader, gmf_criterion, device)
    print(f"📊 Fold {i}: GMF: Loss={loss:.4f}, RMSE={rmse:.4f}")
    torch.save(gmf.state_dict(), f"./pretrain/gmf_fold{i}.pt")

    # === PRETRAIN MLP ===
    mlp = MLP(n_users, n_items, mlp_layers).to(device)
    mlp_optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)
    mlp_criterion = nn.MSELoss()

    print("🔧 Pretraining MLP...")
    train(mlp, train_loader, mlp_criterion, mlp_optimizer, device)
    loss, rmse = evaluation(mlp, test_loader, mlp_criterion, device)
    print(f"📊 Fold {i}: MLP: Loss={loss:.4f}, RMSE={rmse:.4f}")
    torch.save(mlp.state_dict(), f"./pretrain/mlp_fold{i}.pt")

    # === LOAD PRETRAINED & BUILD NCF ===
    pretrained_gmf = GMF(n_users, n_items, gmf_factors, for_NeuMF=False).to(device)
    pretrained_mlp = MLP(n_users, n_items, mlp_layers, for_NeuMF=False).to(device)
    pretrained_gmf.load_state_dict(torch.load(f"./pretrain/gmf_fold{i}.pt"))
    pretrained_mlp.load_state_dict(torch.load(f"./pretrain/mlp_fold{i}.pt"))

    ncf = NeuralCF(n_users, n_items, gmf_factors, mlp_layers,
                   GMF_model=pretrained_gmf, MLP_model=pretrained_mlp).to(device)

    ncf_criterion = nn.MSELoss()
    ncf_optimizer = torch.optim.Adam(ncf.parameters(), lr=learning_rate)

    print("🧠 Fine-tuning NeuMF (NCF)...")
    train(ncf, train_loader, ncf_criterion, ncf_optimizer, device)

    loss, rmse = evaluation(ncf, test_loader, ncf_criterion, device)
    print(f"📊 Fold {i}: Loss={loss:.4f}, RMSE={rmse:.4f}")
